<a href="https://colab.research.google.com/github/signalcompose/MusicTransformer-tensorflow2.0/blob/master/GoogleCoraboratory/MusicTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###前処理
* Music Transformer のリポジトリダウンロード
* リポジトリのサブモジュールをダウンロード
* [将来削除予定]経過時間を表示するブランチにチェックアウトする
* Yamaha e-piano competition のデータダウンロード
* preprocess に必要なモジュールのインストール
* preprocess.py の実行

In [0]:
!git clone https://github.com/signalcompose/MusicTransformer-tensorflow2.0.git 
%cd MusicTransformer-tensorflow2.0/
!git submodule init && git submodule update
!git checkout -b feature/PrintTrainTimer origin/feature/PrintTrainTimer
!mkdir -p dataset/ecomp/midi
!sh dataset/scripts/ecomp_piano_downloader.sh dataset/ecomp/midi
!mkdir -p dataset/ecomp/processed
!pip install progress pretty_midi config
!python preprocess.py dataset/ecomp/midi dataset/ecomp/processed

## トレーニング


In [0]:
!python train.py --epochs=1 --batch_size=2 --save_path=dataset/ecomp/model --pickle_dir=dataset/ecomp/processed